In [ ]:
import dask.bag as db
import dask
from dask.diagnostics import ProgressBar

In [ ]:
import xarray as xr
import fsspec
import glob
xr.set_options(display_style='text');

In [ ]:
import s3fs
s3 = s3fs.S3FileSystem(anon=False)

In [ ]:
references = [f's3://imos-data-pixeldrill/zarrs/{year}/'for year in range(1992, 2000)]
references

In [ ]:
b = db.from_sequence(references)
zarrs = b.map(xr.open_zarr)

In [ ]:
%%time
with dask.config.set(scheduler='threads'):
    with ProgressBar():
        zarrs = zarrs.compute()

In [ ]:
%%time
ds = xr.concat(zarrs,dim='time',coords='minimal',compat='override',combine_attrs='override', fill_value='')

In [ ]:
%%time
timeseries = ds.sel(lat=-20.01,lon=118.6,method='nearest').compute()

In [ ]:
timeseries.sea_surface_temperature.plot()

## Convert To NetCDF

In [ ]:
%%time
timeseries.to_netcdf('timeseries.nc')